In [92]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import statsmodels.api as sm
import seaborn as sns 


In [93]:
jobs = pd.read_csv('../data/fullsample.csv', nrows=1000000)
jobs

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,31378652,COMPLETED,2021-08-17T15:18:31,2021-08-17T15:18:49,45056Mn,0,3-00:00:00,00:00:18,1,3,production,0:0
999996,31378653,COMPLETED,2021-08-17T15:05:24,2021-08-17T15:05:44,45056Mn,0,23:00:00,00:00:20,1,8,production,0:0
999997,31378654,COMPLETED,2021-08-17T15:07:24,2021-08-17T15:07:38,45056Mn,0,23:00:00,00:00:14,1,8,production,0:0
999998,31378655,COMPLETED,2021-08-17T15:07:24,2021-08-17T15:07:41,45056Mn,0,23:00:00,00:00:17,1,8,production,0:0


In [94]:
jobs = jobs.drop(jobs[jobs['END'] == 'Unknown'].index)
jobs = jobs[jobs['EXITCODE'] == '0:0']
jobs.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17T07:45:07,2021-09-06T16:17:34,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0


In [95]:
jobs['END'] = pd.to_datetime(jobs['END'])
jobs.info()
jobs

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997220 entries, 1 to 999999
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   JOBID      997220 non-null  object        
 1   STATE      997220 non-null  object        
 2   BEGIN      997220 non-null  object        
 3   END        997220 non-null  datetime64[ns]
 4   REQMEM     997220 non-null  object        
 5   USEDMEM    997220 non-null  object        
 6   REQTIME    997220 non-null  object        
 7   USEDTIME   997220 non-null  object        
 8   NODES      997220 non-null  int64         
 9   CPUS       997220 non-null  int64         
 10  PARTITION  997220 non-null  object        
 11  EXITCODE   997220 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 98.9+ MB


,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17T07:45:07,2021-09-06 16:17:34,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,31378652,COMPLETED,2021-08-17T15:18:31,2021-08-17 15:18:49,45056Mn,0,3-00:00:00,00:00:18,1,3,production,0:0
999996,31378653,COMPLETED,2021-08-17T15:05:24,2021-08-17 15:05:44,45056Mn,0,23:00:00,00:00:20,1,8,production,0:0
999997,31378654,COMPLETED,2021-08-17T15:07:24,2021-08-17 15:07:38,45056Mn,0,23:00:00,00:00:14,1,8,production,0:0
999998,31378655,COMPLETED,2021-08-17T15:07:24,2021-08-17 15:07:41,45056Mn,0,23:00:00,00:00:17,1,8,production,0:0


In [96]:
jobs['REQMEM'] = jobs['REQMEM'].str.rstrip('Mnc')

jobs['REQMEM'] = pd.to_numeric(jobs['REQMEM'])
jobs

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05 11:36:32,262144,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05 19:04:53,204800,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07 20:52:55,65536,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10 16:45:24,16384,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17T07:45:07,2021-09-06 16:17:34,16384,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,31378652,COMPLETED,2021-08-17T15:18:31,2021-08-17 15:18:49,45056,0,3-00:00:00,00:00:18,1,3,production,0:0
999996,31378653,COMPLETED,2021-08-17T15:05:24,2021-08-17 15:05:44,45056,0,23:00:00,00:00:20,1,8,production,0:0
999997,31378654,COMPLETED,2021-08-17T15:07:24,2021-08-17 15:07:38,45056,0,23:00:00,00:00:14,1,8,production,0:0
999998,31378655,COMPLETED,2021-08-17T15:07:24,2021-08-17 15:07:41,45056,0,23:00:00,00:00:17,1,8,production,0:0


In [97]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997220 entries, 1 to 999999
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   JOBID      997220 non-null  object        
 1   STATE      997220 non-null  object        
 2   BEGIN      997220 non-null  object        
 3   END        997220 non-null  datetime64[ns]
 4   REQMEM     997220 non-null  int64         
 5   USEDMEM    997220 non-null  object        
 6   REQTIME    997220 non-null  object        
 7   USEDTIME   997220 non-null  object        
 8   NODES      997220 non-null  int64         
 9   CPUS       997220 non-null  int64         
 10  PARTITION  997220 non-null  object        
 11  EXITCODE   997220 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 98.9+ MB


In [98]:
jobs['REQMEMTOT'] = (jobs['REQMEM'])*(jobs['NODES'])

jobs['REQMEMPERCORE'] = jobs['REQMEMTOT']/jobs['CPUS']
jobs

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,REQMEMTOT,REQMEMPERCORE
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05 11:36:32,262144,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0,262144,262144.000000
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05 19:04:53,204800,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0,204800,6400.000000
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07 20:52:55,65536,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0,65536,8192.000000
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10 16:45:24,16384,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0,16384,16384.000000
5,31364111_3,COMPLETED,2021-08-17T07:45:07,2021-09-06 16:17:34,16384,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0,16384,16384.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,31378652,COMPLETED,2021-08-17T15:18:31,2021-08-17 15:18:49,45056,0,3-00:00:00,00:00:18,1,3,production,0:0,45056,15018.666667
999996,31378653,COMPLETED,2021-08-17T15:05:24,2021-08-17 15:05:44,45056,0,23:00:00,00:00:20,1,8,production,0:0,45056,5632.000000
999997,31378654,COMPLETED,2021-08-17T15:07:24,2021-08-17 15:07:38,45056,0,23:00:00,00:00:14,1,8,production,0:0,45056,5632.000000
999998,31378655,COMPLETED,2021-08-17T15:07:24,2021-08-17 15:07:41,45056,0,23:00:00,00:00:17,1,8,production,0:0,45056,5632.000000


In [99]:
td_converts=['REQTIME', 'USEDTIME']

for col in td_converts:
    jobs[col] = pd.to_timedelta(jobs[col].str.replace('-', 'days '))
    
jobs

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,REQMEMTOT,REQMEMPERCORE
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05 11:36:32,262144,20604.62M,30 days 00:00:00,30 days 00:00:23,1,1,cgw-platypus,0:0,262144,262144.000000
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05 19:04:53,204800,57553.77M,30 days 00:00:00,30 days 00:00:14,1,32,cgw-tbi01,0:0,204800,6400.000000
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07 20:52:55,65536,20577.96M,29 days 04:00:00,29 days 04:00:04,1,8,cgw-platypus,0:0,65536,8192.000000
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10 16:45:24,16384,9733.43M,24 days 09:00:00,24 days 09:00:17,1,1,production,0:0,16384,16384.000000
5,31364111_3,COMPLETED,2021-08-17T07:45:07,2021-09-06 16:17:34,16384,9708.04M,24 days 09:00:00,20 days 08:32:27,1,1,production,0:0,16384,16384.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,31378652,COMPLETED,2021-08-17T15:18:31,2021-08-17 15:18:49,45056,0,3 days 00:00:00,0 days 00:00:18,1,3,production,0:0,45056,15018.666667
999996,31378653,COMPLETED,2021-08-17T15:05:24,2021-08-17 15:05:44,45056,0,0 days 23:00:00,0 days 00:00:20,1,8,production,0:0,45056,5632.000000
999997,31378654,COMPLETED,2021-08-17T15:07:24,2021-08-17 15:07:38,45056,0,0 days 23:00:00,0 days 00:00:14,1,8,production,0:0,45056,5632.000000
999998,31378655,COMPLETED,2021-08-17T15:07:24,2021-08-17 15:07:41,45056,0,0 days 23:00:00,0 days 00:00:17,1,8,production,0:0,45056,5632.000000


In [100]:
code to determine concurrent processes:
begin_load_df = fs_df[fs_df['PARTITION'] == 'production'][['BEGIN']].rename(columns={'BEGIN':'TIME'})
begin_load_df['process'] = 1
end_load_df = fs_df[fs_df['PARTITION'] == 'production'][['END']].rename(columns={'END':'TIME'})
end_load_df['process'] = -1
load_df = begin_load_df.append(end_load_df)

resample_interval = "D"
fs_ts = pd.Series(list(load_df['process']), index=list(load_df['TIME']))
fs_resample = fs_ts.resample(resample_interval).sum().to_frame().cumsum()

plt.figure(figsize=(15, 10))
plt.plot(fs_resample.index, fs_resample[0])


SyntaxError: invalid syntax (<ipython-input-100-66176c1141ab>, line 1)